### Carga de datos, etc


In [14]:
import os
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cv2
import PIL.Image as Image

from skimage import io, color, transform
from skimage.feature import hog

import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report

In [17]:
gray_matrix = np.load('DatosDF\gray_matrix.npy', allow_pickle=True)
pca = np.load('DatosDF\pca.npy', allow_pickle=True)
estados = np.load('DatosDF\estados.npy', allow_pickle=True)
label = np.load('DatosDF\labels.npy', allow_pickle=True)

<>:1: SyntaxWarning: invalid escape sequence '\g'
<>:2: SyntaxWarning: invalid escape sequence '\p'
<>:3: SyntaxWarning: invalid escape sequence '\e'
<>:4: SyntaxWarning: invalid escape sequence '\l'
<>:1: SyntaxWarning: invalid escape sequence '\g'
<>:2: SyntaxWarning: invalid escape sequence '\p'
<>:3: SyntaxWarning: invalid escape sequence '\e'
<>:4: SyntaxWarning: invalid escape sequence '\l'
C:\Users\gonza\AppData\Local\Temp\ipykernel_14624\3099182347.py:1: SyntaxWarning: invalid escape sequence '\g'
  gray_matrix = np.load('DatosDF\gray_matrix.npy', allow_pickle=True)
C:\Users\gonza\AppData\Local\Temp\ipykernel_14624\3099182347.py:2: SyntaxWarning: invalid escape sequence '\p'
  pca = np.load('DatosDF\pca.npy', allow_pickle=True)
C:\Users\gonza\AppData\Local\Temp\ipykernel_14624\3099182347.py:3: SyntaxWarning: invalid escape sequence '\e'
  estados = np.load('DatosDF\estados.npy', allow_pickle=True)
C:\Users\gonza\AppData\Local\Temp\ipykernel_14624\3099182347.py:4: SyntaxWarning:

In [18]:
df = pd.DataFrame({'gray_matrix': gray_matrix,'pca': pca,'estados': estados,'label': label})

In [19]:
dfxusar = df[df['estados'] == 1]
elresto = df[df['estados'] == 0]
dfxusar

,gray_matrix,pca,estados,label
16,"[[175, 166, 185, 198, 202, 213, 212, 209, 210,...","[1.0912320462992262, 0.3657536288748723, 0.283...",1,angry
23,"[[26, 29, 30, 35, 45, 60, 108, 145, 161, 161, ...","[-0.7706739587282956, 0.721693515528713, 1.215...",1,angry
30,"[[17, 18, 19, 19, 17, 15, 16, 17, 18, 20, 18, ...","[0.7941313890964633, -0.6699144967452324, 0.43...",1,angry
31,"[[0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 0, 1, 13, 37, ...","[1.5112234099411108, 1.0339331627332178, -0.16...",1,angry
45,"[[254, 254, 254, 254, 254, 254, 254, 254, 254,...","[1.0902360530996267, 0.01650497232340127, -0.2...",1,angry
...,...,...,...,...
35863,"[[9, 38, 69, 99, 86, 102, 111, 105, 65, 58, 64...","[-0.5491822800503183, 0.37287554580139537, 0.5...",1,surprise
35869,"[[253, 253, 254, 253, 170, 101, 105, 102, 85, ...","[-0.831189505684772, -0.9824639195355812, -0.5...",1,surprise
35875,"[[252, 247, 146, 93, 94, 89, 99, 94, 93, 83, 9...","[-1.6612481070642713, -0.7122384048718365, 0.9...",1,surprise
35878,"[[33, 44, 49, 40, 35, 28, 23, 14, 9, 16, 17, 5...","[0.4970099444085885, -0.026777335969787118, 0....",1,surprise


In [20]:
X = np.array(dfxusar['pca'].values.tolist())
y = dfxusar['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19011)

## Redes Neuronales

In [32]:
mlp = MLPClassifier(hidden_layer_sizes=(100,),
                    activation='relu',
                    solver='sgd',
                    alpha=0,
                    learning_rate='constant',
                    max_iter=10000,
                    tol=1e-6,
                    verbose=True,
                    early_stopping=True,
                    validation_fraction=0.2,
                    momentum=0,
                    n_iter_no_change=25,
                    random_state=42)
mlp.fit(X_train, y_train)

Iteration 1, loss = 1.95697770
Validation score: 0.172087
Iteration 2, loss = 1.95439576
Validation score: 0.174797
Iteration 3, loss = 1.95182563
Validation score: 0.177507
Iteration 4, loss = 1.94926973
Validation score: 0.182927
Iteration 5, loss = 1.94672798
Validation score: 0.185637
Iteration 6, loss = 1.94418900
Validation score: 0.191057
Iteration 7, loss = 1.94165973
Validation score: 0.193767
Iteration 8, loss = 1.93913808
Validation score: 0.195122
Iteration 9, loss = 1.93663051
Validation score: 0.203252
Iteration 10, loss = 1.93412671
Validation score: 0.208672
Iteration 11, loss = 1.93164280
Validation score: 0.210027
Iteration 12, loss = 1.92916319
Validation score: 0.210027
Iteration 13, loss = 1.92669064
Validation score: 0.212737
Iteration 14, loss = 1.92423325
Validation score: 0.214092
Iteration 15, loss = 1.92178155
Validation score: 0.219512
Iteration 16, loss = 1.91933711
Validation score: 0.224932
Iteration 17, loss = 1.91690366
Validation score: 0.230352
Iterat

MLPClassifier(alpha=0, early_stopping=True, max_iter=10000, momentum=0,
              n_iter_no_change=25, random_state=42, solver='sgd', tol=1e-06,
              validation_fraction=0.2, verbose=True)

In [33]:
y_pred_e= mlp.predict(X_train)
accuracy = accuracy_score(y_train, y_pred_e)
print(f'Precisión del modelo entrenamiento: {accuracy}')

print(classification_report(y_train, y_pred_e))

Precisión del modelo entrenamiento: 0.7125813449023861
              precision    recall  f1-score   support

       angry       0.62      0.54      0.57       462
     disgust       0.63      0.29      0.40       435
        fear       0.65      0.59      0.62       579
       happy       0.79      0.99      0.88       650
     neutral       0.74      0.86      0.79       622
         sad       0.66      0.72      0.69       467
    surprise       0.78      0.85      0.81       473

    accuracy                           0.71      3688
   macro avg       0.69      0.69      0.68      3688
weighted avg       0.70      0.71      0.70      3688



In [34]:
# Realizar predicciones en el conjunto de prueba
y_pred = mlp.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo de tsteo: {accuracy}')

print(classification_report(y_test, y_pred))

Precisión del modelo de tsteo: 0.7009750812567714
              precision    recall  f1-score   support

       angry       0.60      0.55      0.57       101
     disgust       0.60      0.30      0.40       112
        fear       0.64      0.60      0.62       156
       happy       0.78      0.97      0.87       149
     neutral       0.71      0.85      0.77       153
         sad       0.67      0.70      0.69       119
    surprise       0.79      0.80      0.79       133

    accuracy                           0.70       923
   macro avg       0.68      0.68      0.67       923
weighted avg       0.69      0.70      0.69       923



## [Conjunto Pequeño] GridSearch - Optimizacion de parametros

### Primero

In [ ]:

mlp = MLPClassifier(hidden_layer_sizes=(100,),
                    activation='logistic',
                    solver='sgd',
                    alpha=0,
                    learning_rate='constant',
                    max_iter=10000,
                    tol=1e-4,
                    verbose=True,
                    momentum=0,
                    n_iter_no_change=25,
                    random_state=42)

param_grid = {
    'learning_rate_init': [0.2,0.1,0.01],
    'batch_size': [32,128,512],
    'activation' : ['identity', 'tanh', 'relu'],
    'hidden_layer_sizes' : [(200,), (410,), (821,), (200, 200), (410, 410), (821, 821)]
}

grid_search = GridSearchCV(estimator=mlp, param_grid=param_grid, cv=5, scoring='accuracy', verbose=10,n_jobs=-1,return_train_score = True)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print(f"Mejores hiperparámetros: {best_params}")

best_rf_classifier = grid_search.best_estimator_
best_rf_classifier.fit(X_train, y_train)

y_pred = best_rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo: {accuracy}')

print(classification_report(y_test, y_pred))

**Resultados** 

Precisión del modelo: 0.8266522210184182
- 'activation': 'relu', 
- 'batch_size': 32, 
- 'hidden_layer_sizes': (150, 150), 
- 'learning_rate_init': 0.1